In [6]:
!pip install zenrows

  Obtaining dependency information for zenrows from https://files.pythonhosted.org/packages/3c/0a/744bd008947c48d64aaab53799bbae6c6f884142faf197e27641b4bb69c9/zenrows-1.3.2-py3-none-any.whl.metadata


In [1]:
from zenrows import ZenRowsClient
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import csv
import os

In [60]:
#경로 지정
url = 'https://war.ukraine.ua/articles/why-should-you-not-consider-ukrainians-and-russians-as-one-nation-here-are-six-points-from-a-historical-and-a-modern-perspective/'

apikey = 'c5884a526eb627b7e9aef2dcc3c84d3fe74f98cd'
params = {
    'url': url,
    'apikey': apikey,
    'js_render': 'true'
}
response = requests.get('https://api.zenrows.com/v1/', params=params)

In [61]:
# CSV 파일을 생성하고 헤더를 작성합니다.
with open('extracted_data187.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Date', 'Text'])
    
    # BeautifulSoup을 사용하여 HTML 내용을 파싱합니다.
    soup = BeautifulSoup(response.text, 'html.parser')

    # <h1> 태그에서 제목을 추출합니다.
    title = soup.find('h1').get_text().strip() if soup.find('h1') else 'No Title'

    # <div class="date"> 태그에서 날짜를 추출합니다.
    date_div = soup.find('div', class_='post-date')
    date = date_div.get_text().strip() if date_div else 'No Date'

    # <p> 태그들의 내용을 하나의 문자열로 합칩니다.
    p_tags = soup.find_all('p')
    text = ' '.join(p.get_text().strip() for p in p_tags)

    # 추출한 데이터를 CSV 파일에 작성합니다.
    writer.writerow([title, date, text])

In [62]:
import pandas as pd

data_frame = []
path = os.getcwd()
file_list = os.listdir(path)

for file_lists in file_list:
    if file_lists.endswith('.csv'):
        full_path = os.path.join(path, file_lists)
        data_frame.append(pd.read_csv(full_path))

df = pd.concat(data_frame)
df

,Title,Date,Text
0,The Russian military launched an airstrike on ...,12/03/2022,Russian occupation forces dropped several bomb...
0,Russia attacked Pokrovsk with two Iskander mis...,08/08/2023,Russian forces launched two missile strikes o...
0,The Russian occupiers kidnapped the mayor of M...,11/03/2022,The Russian occupiers abducted Mayor Ivan Fed...
0,"Mykyta, along with 105 Ukrainian children, was...",20/10/2023,"Mykyta Bilanchuk, 9, was illegally taken by t..."
0,"Artem, a Ukrainian teenage boy, was deported t...",29/06/2023,Russian forces forcibly deported sixteen-year...
...,...,...,...
0,15 servicewomen returned from Russian captivit...,02/04/2022,Russians forced captured Ukrainian women to un...
0,The Russians have taken hostage the elderly fa...,23/03/2022,The Russians kidnapped the 75-year-old father ...
0,"In the Sumy region, the Russian occupiers kidn...",21/03/2022,Journalists of the Ukrainian public broadcaste...
0,The Russian military took doctors and patients...,14/03/2022,"On March 14, Russian troops seized the Mariup..."


In [63]:
df.to_csv('concated_ukrain_warnews.csv', index=False, encoding='utf-8-sig')